<a href="https://colab.research.google.com/github/EriTrn/My_Project/blob/main/Analisis_sentimen_apk_Byu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install google-play-scraper
!pip install sastrawi
!pip install requests

In [59]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store serta library yang dibutuhkan

import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import string
import csv
import requests
from google_play_scraper import app, reviews, Sort, reviews_all
from io import StringIO
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud

In [7]:
pd.options.mode.chained_assignment = None
seed = 0
np.random.seed(seed)
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.

scrapreview = reviews_all(
    'com.byu.id',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000
)

In [9]:
# Menyimpan ulasan dalam file csv

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])
    for review in scrapreview:
        writer.writerow([review['content']])

In [10]:
# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview)

In [11]:
# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

In [12]:
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('ulasan_aplikasi.csv', index=False)

In [13]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148717 entries, 0 to 148716
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              148717 non-null  object        
 1   userName              148717 non-null  object        
 2   userImage             148717 non-null  object        
 3   content               148716 non-null  object        
 4   score                 148717 non-null  int64         
 5   thumbsUpCount         148717 non-null  int64         
 6   reviewCreatedVersion  125301 non-null  object        
 7   at                    148717 non-null  datetime64[ns]
 8   replyContent          130470 non-null  object        
 9   repliedAt             130470 non-null  datetime64[ns]
 10  appVersion            125301 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 12.5+ MB


In [14]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()

In [15]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

In [16]:
# Membersihkan data

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')
    return text

In [17]:
# Mengubah semua karakter dalam teks menjadi huruf kecil

def casefoldingText(text):
    text = text.lower()
    return text

In [18]:
# Memecah atau membagi string, teks menjadi daftar token

def tokenizingText(text):
    text = word_tokenize(text)
    return text

In [19]:
# Menghapus stopwords dalam teks

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

In [20]:
# Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata

def stemmingText(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    words = text.split()

    stemmed_words = [stemmer.stem(word) for word in words]

    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

In [21]:
# Mengubah daftar kata menjadi kalimat

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [22]:
# Menghapus slang words atau kata-kata informal
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [23]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

In [24]:
# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

In [25]:
# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

In [26]:
# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

In [27]:
# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

In [28]:
# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [34]:
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

In [43]:
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])

else:
    print("Failed to fetch positive lexicon data")

In [44]:
# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

In [45]:
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')

    for row in reader:
        lexicon_negative[row[0]] = int(row[1])

else:
    print("Failed to fetch negative lexicon data")

In [46]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text:
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]

    for word in text:
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]

    polarity=''

    if (score >= 0):
        polarity = 'positive'
    elif (score < 0):
        polarity = 'negative'

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity

In [47]:
# Menampikan Hasil polaritas

results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    65227
negative    44262
Name: count, dtype: int64


In [48]:
# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']

In [51]:
# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

In [52]:
# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [53]:
# Menampilkan hasil ekstraksi fitur
features_df

,aja,aktivasi,aman,aneh,apk,aplikasi,aplikasinya,app,bagus,banget,...,turun,udah,udh,ulang,unlimited,update,via,wifi,yg,youtube
0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.188929,0.00000,0.129817,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.154133,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.53377,0.000000,0.00000,0.119917,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.242739,0.0,0.0,0.0,0.0,0.00000,0.000000,0.18997,0.212809,0.228647,...,0.0,0.000000,0.162444,0.0,0.0,0.0,0.0,0.0,0.126335,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.14564,0.190473,0.00000,0.261757,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.268646,0.0,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.253050,...,0.0,0.141024,0.000000,0.0,0.0,0.0,0.0,0.0,0.139818,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109484,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
109485,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.000000,0.0
109486,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
109487,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [56]:
# Membagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [60]:
# Membuat, melatih, memprediksi, mengevaluasi menggunakan metode Naive Bayes
naive_bayes = BernoulliNB()

naive_bayes.fit(X_train.toarray(), y_train)

y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)

Naive Bayes - accuracy_train: 0.783014236622484
Naive Bayes - accuracy_test: 0.7839072061375468


In [ ]:
# Jika menggunakan random forest
# # Membuat, melatih, memprediksi, mengevaluasi menggunakan metode random forest

# random_forest = RandomForestClassifier()

# random_forest.fit(X_train.toarray(), y_train)

# y_pred_train_rf = random_forest.predict(X_train.toarray())
# y_pred_test_rf = random_forest.predict(X_test.toarray())

# accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
# accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# print('Random Forest - accuracy_train:', accuracy_train_rf)
# print('Random Forest - accuracy_test:', accuracy_test_rf)

In [ ]:
# jika menggunakan metode logistic regression
# # Membuat, melatih, memprediksi, mengevaluasi menggunakan metode logistic regresion

# logistic_regression = LogisticRegression()

# logistic_regression.fit(X_train.toarray(), y_train)

# y_pred_train_lr = logistic_regression.predict(X_train.toarray())
# y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)
# accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

# print('Logistic Regression - accuracy_train:', accuracy_train_lr)
# print('Logistic Regression - accuracy_test:', accuracy_test_lr)

In [ ]:
# # jika menggunakan metode decision tree
# # Membuat, melatih, memprediksi, mengevaluasi menggunakan metode decision tree

# decision_tree = DecisionTreeClassifier()

# decision_tree.fit(X_train.toarray(), y_train)

# y_pred_train_dt = decision_tree.predict(X_train.toarray())
# y_pred_test_dt = decision_tree.predict(X_test.toarray())

# accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
# accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

# print('Decision Tree - accuracy_train:', accuracy_train_dt)
# print('Decision Tree - accuracy_test:', accuracy_test_dt)